# I-Con Playground

**A Student-Friendly Tool for Exploring I-Con Representation Learning**

---

## What is I-Con?

**I-Con (Integrated Contrastive learning)** is a unifying framework for representation learning that shows how many seemingly different methods are actually optimizing the same underlying objective: minimizing the KL divergence between two neighborhood distributions.

This framework unifies over 20 methods including:
- **Dimensionality reduction**: SNE, t-SNE
- **Contrastive learning**: SimCLR, InfoNCE
- **Clustering**: k-means, spectral clustering
- **Supervised learning**: Cross-entropy classification

The key insight is that all these methods can be written as:

$$\mathcal{L} = D_{KL}(P \| Q)$$

Where:
- $P$ is the **supervisory distribution** (defines which points should be neighbors)
- $Q$ is the **learned distribution** (defines similarity in the embedding space)

---

## About This Notebook

This notebook is a **playground** for running small I-Con experiments on datasets like CIFAR-10 and MNIST. It is designed for:
- **Learning**: Understand how different I-Con configurations affect learned representations
- **Experimentation**: Quickly try different backbones, objectives, and hyperparameters
- **Visualization**: See how embeddings cluster and separate by class

**Note**: This is NOT designed for reproducing the full I-Con paper experiments (e.g., ImageNet-scale training). For that, refer to the main I-Con repository documentation.

---

## Setup

First, let's import the necessary modules from the playground.

In [ ]:
# Add the parent directories to the path (run this if importing fails)
import sys
sys.path.insert(0, '../..')  # Add root of I-Con repo
sys.path.insert(0, '..')     # Add playground directory

# Core playground imports
from playground.playground_config import PlaygroundConfig, quick_config
from playground.playground_runner import run_playground_experiment, load_experiment_results
from playground.playground_viz import plot_training_curves, plot_embeddings_2d, create_experiment_summary
from playground.playground_probes import run_linear_probe, run_toy_negation_probe, analyze_embedding_separability

# Standard libraries
import numpy as np
import matplotlib.pyplot as plt

# For nicer notebook display
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100

print("Imports successful!")

## Step 1: Configure Your Experiment

The `PlaygroundConfig` class lets you easily set up an I-Con experiment. Here are the key parameters:

| Parameter | Description | Options |
|-----------|-------------|----------|
| `dataset` | Dataset to train on | `cifar10`, `cifar100`, `mnist`, `stl10` |
| `backbone` | Encoder architecture | `resnet18`, `resnet34`, `resnet50`, `simplecnn`, `mlp` |
| `icon_mode` | I-Con objective preset | `simclr_like`, `sne_like`, `tsne_like`, `supervised`, `cluster_like` |
| `epochs` | Training epochs | Any positive integer (start small, e.g., 5-20) |
| `batch_size` | Batch size | 64, 128, 256, etc. |
| `temperature` | Softmax temperature | 0.1 - 1.0 (lower = sharper) |
| `embedding_dim` | Size of learned embeddings | 32, 64, 128, 256, etc. |

### I-Con Mode Descriptions

- **`simclr_like`**: Contrastive learning (InfoNCE-style) - learns by pulling augmented views together
- **`sne_like`**: SNE-style embedding with Gaussian kernel in embedding space
- **`tsne_like`**: t-SNE-style with Student-t kernel (heavier tails, better for visualization)
- **`supervised`**: Uses class labels directly as the supervisory signal
- **`cluster_like`**: Clustering-oriented with learnable temperature

In [ ]:
# Create a configuration for your experiment
# Feel free to modify these parameters!

config = PlaygroundConfig(
    # Dataset and model
    dataset="cifar10",           # Try: "cifar10", "mnist", "cifar100"
    backbone="resnet18",         # Try: "resnet18", "resnet50" (use "simplecnn" for MNIST)
    icon_mode="simclr_like",     # Try: "simclr_like", "tsne_like", "supervised"
    
    # Training hyperparameters
    epochs=10,                   # Start small (5-10), increase if needed
    batch_size=256,              # Reduce if you run out of memory
    learning_rate=1e-3,          # Standard starting point
    temperature=0.5,             # Lower = sharper distributions
    embedding_dim=128,           # Size of the representation
    
    # Output location
    output_dir="playground_runs",
)

# Print the configuration summary
print(config.describe())

### Alternative: Use a Preset

If you just want to get started quickly, you can use a preset configuration:

In [ ]:
# Uncomment one of these to use a preset instead:

# config = quick_config("cifar_contrastive", epochs=10)  # SimCLR-like on CIFAR-10
# config = quick_config("cifar_supervised", epochs=10)   # Supervised on CIFAR-10  
# config = quick_config("mnist_tsne", epochs=10)         # t-SNE-like on MNIST

# print(config.describe())

## Step 2: Run the Experiment

Now let's train the model! This will:
1. Load the dataset
2. Initialize the encoder and I-Con objective
3. Train for the specified number of epochs
4. Extract embeddings from the validation set
5. Save everything to the output directory

**Note**: Training time depends on your hardware. With a GPU, CIFAR-10 with ResNet-18 for 10 epochs should take 5-15 minutes.

In [ ]:
# Run the experiment!
# Set verbose=True to see progress, verbose=False for less output

results = run_playground_experiment(
    config,
    verbose=True,
    gpu=True,  # Set to False if you don't have a GPU
)

print(f"\nExperiment complete! Results saved to: {results['paths']['run_dir']}")

## Step 3: Visualize Training Progress

Let's look at how the loss evolved during training.

In [ ]:
# Plot training curves
fig = plot_training_curves(
    results["logs"],
    figsize=(12, 4),
    show=True,
)

# Print final metrics
if results["logs"]["val_losses"]:
    print(f"Final validation loss: {results['logs']['val_losses'][-1]:.4f}")
if results["logs"]["val_accuracies"]:
    print(f"Final validation accuracy: {results['logs']['val_accuracies'][-1]:.4f}")

## Step 4: Visualize the Learned Embeddings

One of the most interesting things to look at is how the learned representations organize samples in space. We'll use dimensionality reduction (PCA or t-SNE) to project the high-dimensional embeddings to 2D.

**What to look for:**
- Do points of the same class cluster together?
- How well separated are different classes?
- Are there any interesting sub-clusters within classes?

In [ ]:
# Visualize embeddings with PCA (fast)
print(f"Embedding shape: {results['embeddings'].shape}")
print(f"Number of samples: {len(results['labels'])}")
print(f"Number of classes: {len(np.unique(results['labels']))}")

fig = plot_embeddings_2d(
    results["embeddings"],
    results["labels"],
    method="pca",  # Fast! Use "tsne" for nicer (but slower) visualization
    title=f"Learned Embeddings ({config.icon_mode})",
    figsize=(10, 8),
    show=True,
)

In [ ]:
# Optional: Visualize with t-SNE (takes longer but often looks nicer)
# Uncomment the lines below to run

# fig = plot_embeddings_2d(
#     results["embeddings"],
#     results["labels"],
#     method="tsne",
#     title=f"Learned Embeddings ({config.icon_mode}) - t-SNE",
#     max_samples=2000,  # Limit samples for speed
#     perplexity=30,
#     show=True,
# )

## Step 5: Evaluate with Linear Probe

A **linear probe** trains a simple logistic regression classifier on top of the frozen embeddings. This is a standard way to evaluate representation quality:

- **High accuracy** = embeddings are linearly separable by class = good representations
- **Low accuracy** = class information is not easily accessible = embeddings may need more training or different objective

In [ ]:
# Run linear probe evaluation
probe_results = run_linear_probe(
    results["embeddings"],
    results["labels"],
    test_size=0.2,
    verbose=True,
)

print(f"\nLinear probe test accuracy: {probe_results['test_accuracy']:.2%}")

In [ ]:
# Analyze embedding separability
# This shows how well different classes are separated in the embedding space

sep_results = analyze_embedding_separability(
    results["embeddings"],
    results["labels"],
    verbose=True,
)

## Step 6: Create a Summary

Let's create a summary figure that shows everything at a glance.

In [ ]:
# Create a summary figure
fig = create_experiment_summary(
    results,
    show=True,
)

---

## Bonus: Toy Negation Probe (Pedagogical)

This is a simple demonstration of **"affirmation bias"** - a phenomenon where embedding models often place semantically opposite sentences close together due to lexical overlap.

For example:
- "The dog is jumping" and "The dog is NOT jumping" share many words
- Naive similarity metrics will score them as very similar
- But they have opposite meanings!

**Note**: This is a TOY example for educational purposes. For rigorous evaluation of negation understanding, see benchmarks like NegBench.

In [ ]:
# Run the toy negation probe
negation_results = run_toy_negation_probe(verbose=True)

---

## Exercises for Students

Now that you've run one experiment, try these exercises to deepen your understanding:

### Exercise 1: Compare I-Con Modes
Run experiments with different `icon_mode` settings and compare:
- How do the embeddings look different between `simclr_like` and `supervised`?
- Which mode gives better linear probe accuracy?

### Exercise 2: Effect of Temperature
Try different `temperature` values (0.1, 0.5, 1.0, 2.0) and observe:
- How does temperature affect training stability?
- How does it affect the clustering in embedding visualizations?

### Exercise 3: Embedding Dimension
Compare different `embedding_dim` values (32, 128, 512):
- Does a larger embedding dimension always help?
- How does it affect the linear probe accuracy?

### Exercise 4: Different Datasets
Try MNIST with `simplecnn` backbone:
- How does the representation differ from CIFAR-10?
- Is it easier or harder to learn good embeddings?

In [ ]:
# Space for your experiments!

# Example: Try a different configuration
# config_new = PlaygroundConfig(
#     dataset="mnist",
#     backbone="simplecnn",
#     icon_mode="tsne_like",
#     epochs=10,
#     temperature=1.0,
# )
# results_new = run_playground_experiment(config_new, verbose=True)

---

## Loading Previous Experiments

If you've already run experiments and want to load them later:

In [ ]:
# Load a previous experiment
# Replace with the actual path to your experiment

# from playground.playground_runner import load_experiment_results, list_experiments

# # List all saved experiments
# experiments = list_experiments("playground_runs")
# for exp in experiments:
#     print(f"{exp['name']}: {exp['dataset']} / {exp['icon_mode']} ({exp['epochs']} epochs)")

# # Load a specific experiment
# loaded_results = load_experiment_results("playground_runs/YOUR_EXPERIMENT_NAME")

---

## References

- **I-Con Paper**: "I-Con: A Unifying Framework for Representation Learning"
- **SimCLR**: Chen et al., "A Simple Framework for Contrastive Learning of Visual Representations"
- **t-SNE**: van der Maaten & Hinton, "Visualizing Data using t-SNE"

---

*This playground is an educational layer on top of the official I-Con implementation. All core I-Con logic comes from the original authors.*